In [1]:
!pip install cassandra-driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 3.2 MB/s eta 0:00:0000:0100:01


In [1]:
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'], port = 9042)

In [2]:
session = cluster.connect()

In [33]:
cluster.metadata.keyspaces

{'system_auth': <cassandra.metadata.KeyspaceMetadata at 0x7fcce9a549d0>,
 'system_schema': <cassandra.metadata.KeyspaceMetadata at 0x7fcccae919d0>,
 'dungeons': <cassandra.metadata.KeyspaceMetadata at 0x7fcccae96b90>,
 'bbdd2': <cassandra.metadata.KeyspaceMetadata at 0x7fcccae97d10>,
 'system_distributed': <cassandra.metadata.KeyspaceMetadata at 0x7fcccae9c350>,
 'system': <cassandra.metadata.KeyspaceMetadata at 0x7fcccae9df10>,
 'system_traces': <cassandra.metadata.KeyspaceMetadata at 0x7fcccaeaa2d0>,
 'system_views': <cassandra.metadata.KeyspaceMetadata at 0x7fcccaeab210>,
 'system_virtual_schema': <cassandra.metadata.KeyspaceMetadata at 0x7fcccaeab3d0>}

In [3]:
session.set_keyspace("dungeons")

In [35]:
print([e for e in session.execute('''DESCRIBE TABLES;''')])

[Row(keyspace_name='dungeons', type='table', name='hall_of_fame'), Row(keyspace_name='dungeons', type='table', name='top_horde'), Row(keyspace_name='dungeons', type='table', name='user_stats')]


## Test the query

In [4]:
import json

def sortFn(dict):
  return dict['n_killed']


def topk_players(country, event_id, k):
    query =f'''SELECT email, user_name, COUNT(*) as n_killed from top_horde WHERE country='{country}' AND event_id={event_id} GROUP BY country, event_id, email;'''
    raw = [{'email': row.email, 'user_name': row.user_name, 'n_killed': row.n_killed} for row in session.execute(query)]
    sorted_data = sorted(raw, key=lambda x: x['n_killed'], reverse=True)
    top_k = json.dumps(sorted_data[:k], indent=4)
    return top_k

result = topk_players('de_DE', 0, 5)
print(result)
    

    

[
    {
        "email": "xullrich@example.org",
        "user_name": "hartmanngloria",
        "n_killed": 19
    },
    {
        "email": "silvesterconradi@example.net",
        "user_name": "bernhardmangold",
        "n_killed": 15
    },
    {
        "email": "gretelholsten@example.com",
        "user_name": "johannekruschwitz",
        "n_killed": 14
    },
    {
        "email": "pero13@example.net",
        "user_name": "zdravko34",
        "n_killed": 14
    },
    {
        "email": "roehrichtpaulina@example.org",
        "user_name": "oderwaldgabriella",
        "n_killed": 14
    }
]


In [6]:
import json


def leaderboard_by_country(country):
    final = []
    for i in range(20):
        to_insert = {}
        
        query = f"SELECT dungeon_name, email, user_name, time_minutes, date from hall_of_fame WHERE country='{country}' AND dungeon_id={i} LIMIT 5;"
        aux = session.execute(query)
        dungeon_id = i
        res = [{'dungeon_name':row.dungeon_name, 'email': row.email, 'user_name': row.user_name,
        'time_minutes': row.time_minutes, 'date': str(row.date)} for row in aux]
        dungeon_name = res[0]['dungeon_name']
        to_insert['dungeon_id'] = i
        to_insert['dungeon_name'] = dungeon_name
        to_insert['Top_5'] = []
        for e in res:
            del e['dungeon_name']
            to_insert['Top_5'].append(e)
            
        final.append(to_insert)
    return json.dumps(final, indent=4)
        

final = leaderboard_by_country('es_ES')
print(final)

[
    {
        "dungeon_id": 0,
        "dungeon_name": "Burghap, Prison of the Jealous Hippies",
        "Top_5": [
            {
                "email": "macario68@example.com",
                "user_name": "julianesther",
                "time_minutes": 0,
                "date": "2022-03-25 05:52:29"
            },
            {
                "email": "begonavera@example.net",
                "user_name": "victorino97",
                "time_minutes": 0,
                "date": "2019-12-17 03:26:00"
            },
            {
                "email": "cbarrena@example.net",
                "user_name": "cecilia28",
                "time_minutes": 0,
                "date": "2019-10-15 15:11:14"
            },
            {
                "email": "vcasals@example.org",
                "user_name": "nestor81",
                "time_minutes": 1,
                "date": "2022-11-25 03:04:31"
            },
            {
                "email": "haydee45@example.net",
         

In [7]:
def user_stats_per_dun(email, dungeon_id):
    final = []
    query = f"select time_minutes, date from user_stats where email='{email}' AND dungeon_id={dungeon_id};"
    aux = session.execute(query)
    for row in aux:
        final.append({'time_minutes':row.time_minutes, 'date':str(row.date)})
        
    return json.dumps(final, indent=4)
result = user_stats_per_dun('macario68@example.com',0)
print(result)

[
    {
        "time_minutes": 17,
        "date": "2022-09-18 18:48:40"
    },
    {
        "time_minutes": 34,
        "date": "2022-06-24 01:07:16"
    },
    {
        "time_minutes": 19,
        "date": "2022-06-11 15:11:25"
    },
    {
        "time_minutes": 27,
        "date": "2022-05-18 20:01:22"
    },
    {
        "time_minutes": 0,
        "date": "2022-03-25 05:52:29"
    },
    {
        "time_minutes": 20,
        "date": "2022-03-16 12:45:05"
    },
    {
        "time_minutes": 40,
        "date": "2022-03-06 02:01:56"
    },
    {
        "time_minutes": 29,
        "date": "2021-11-09 05:14:13"
    },
    {
        "time_minutes": 19,
        "date": "2021-10-11 10:31:03"
    },
    {
        "time_minutes": 46,
        "date": "2021-10-10 01:41:04"
    }
]
